In [4]:
import torch, transformers, json
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)

## Dataset value example 
{'record': 'JAHFAU010000168.1', 'text': 'CTCCTCGGCCAGCAAGCGCATCGTGAGAACCGGACTGACGATCAGCAAAACGGTCGCCAGCGGGCCCAATACCATGCGGCCATCCGGGTTCATCTGGCCGGTCGGCAGGCCAAAGTTAACGTTTAGGGAGAAGATCAGGCCGAGCACCAGGAAGCTGAGGGCGGCGGCCGCGTAAGCCAGCGGCGACACGAAGTAGCGGCGGAACTCGCGCACTGCAATAACCCAGCTGGTGCGCACGATCTAGCTCTCCTGCGGCACGACTTGCTCTTCCTCGCGAGTCAGCAGCAGGAAGATGCTTTCCAGGCTGACCCCGGCCGTTGTCAGCTCCAGCAAGTCCAGATCGGCTTCAACGATGCGGCGCGCCAACAGCGGGCGCAGGTCGCGGCCGGGCGAAGAGATCACCTCCACGGTGCCTGCCTGCCCGGCAAGCTCGACCGACTCCACTCCCGGGACGGACTTGAGCAGGTCGACCAGATCGGCTGCCGGGGCAGAGCTGCGGATCAGAATCCGGTACGCACCGGCCAGCTGGGCCGAAAGATTGGCGGGCGTATCTTCCGCCACCAACCGCCCCTTGTTGATAATCAGGACTCGATCACAAACTTGCTGTGCTTCCGAGAGGATGTGGGTGGACAACATGACGGTCCGCTCCCGCCCGAGCTCTCGGATTAAGTCCCGGACTTCGATGATTTGGGCCGGATCCAGTCCAATGGTCGGCTCGTCCAGGATCAAGACCTCGGGCCGGTGCAAGAGTGCCTGGGCCAGGCCGACCCGCTGTCGCAGACCTTTCGACAGGTTGCCAATCAGGCTATCACCGCGTTTTTCGAGGCCGACCTGCGTTAGGGATTCCTCGACCCGGCCATCCAGGTCCTCCACCCGCCGAAGCGAACCCATGAACGAGAGGTATTCGCGGCCGGACATCTCCGGGTACAGCGGGACGGTCTCCGGCAGGTATCCGACCCGGCGGCGCACCTCGAGCGATTCCTCCACCACGTCGTAGCCGGCGACTTTGGCGCTCCCGCTCGTTGGCGGCATGTATCCGGTGAGGATCCGCATCGTGGTCGTCTTCCCGGCGCCGTTCGGTCCAAGGAATCCCAGGATCTCGCCACGTTTGGCCTCGAAGGAAATTCCCTCGAGCGCGGTCAGGCTCCCGTACGTTTTCGAAAGCTTATTGGCCTCGATCAAGCCTCGGTCCTCAAGCCCAGCTCGCCAGGGGGATGGCCGGACACAAAAACATCCCGGCTCTGCCGTTGCGAAGATCTTGCTACGGAAGACCGGGATGTGAGGGATCCGATCCCGGTGACCACTATACAGAGATTCGCGGCCGGGAGTCAAGACGCAGTTGCGCTTCTAATGAGACGCTAAGCTCGCGGCCCGGTTTTCTTTAAGTTTGCTTGACATGGCATGTGCCAACTTGTACAATCGCCCAGCCTCCCCCACAGGAGCCTTAGAATCGGATGCAAGCTCTGACCGCATTTGAACAAAGCGCGGTTCTCGGCGTTGTCGGGATTGCCATACTGGGGTTGGTCTACGCACTTTTCCTGCGGACCCAGGTGCTGCGCGAACCGGCCGGGTCCCCCCGGATGCAGGAGGTCTGGAATGCGATCCGGGTAGGCGCCGACACTTACCTCCGCCGCCAGCTGGGCAGCATCCTCCCGCTGATCGCGGTCCTCACGGTCGCCCTCTTCCTGAGCGTCTACATCGTCCCCCCCAGCCCGGAGGCGACAGAACGATTCAAAACTTTGGATTCGGACACCGTCCGGCTGGTGATCGGGTTGGGAAGAGCCTTGGCCTTCGTCATGGGGGCCATCTTTAGCGTGACCGTTGGGCAGATTGGGATGCGGATGGCGGTGCAGGCCAATGTTCGAGTCGCCGCGGCGGCCCGCCGGTCGTTCGGGGACTCGCTGCGCATCGCCTACCGGGCCGGCACGATCACCGGGATGTTGACCGACGGACTCGGCCTTTTCGGCGGAACGATGATCTTCATCATCTTCGGAATCGCCTCGCCGGATGCGCTGCTCGGGTTCGGATTCGGCGGGACCCTGGTGGCGCTCTTCATGCGGGTTGGCGGGGGGATCTACACCAAGGCGGCCGACGTCGGGGCGGACCTGGTCGGCAAGATCGAGGCGGGTATCCCCGAGGACGACCCACGCAATCCCGCAGTTATCGCCGATCTGGTGGGCGACAACGTGGGCGACTGCGCCGGGATGGCGGCGGATATCTTTGAGTCATATGAAGTCACGATCGTCTCCGGTCTGATTCTTGGGCTGGCGCTTTGGCACATCACCGGCAACTTCGAATGGGTGGTCTACCCCCTGCTCGTGCGCGGCATCGGTGTGCTTTCTTCGATCATCGGCACCTACGCGGTCCGCGGCGGCCATGGGAAGAGCGGCAATGCGATGCGGGCGATCTTCACCGGGTTCTTGTCCTCGGCGGCGATCTCGGTGGTCTTGTTCGGAATCATGGGATTCTTTTACATGCGAGAGGTGCCGGGAGGCTGGTGGCGGCCGTTCCTGGCCACGACCGCGGGGGTGCTGCTGGCGATCGCCATCGACCGACTGACCGAGTACTTCACCGGAAGCCACGGCACTCCGGTTCAGGAAATCCGCAAGGCCACCAACACCGGCCCGGCGACCACCATTCTCTCCGGGATCGCGGTTGGGTATGAGTCGAGTGTCTGGTCGATCCTGATCATCGCGGTAACCATCTTCGTCTCGATCCTTACTTTCGGCGGAATCTCCGGCATCCCGGCCGGCGAGGCGGTGACCTTTGTGCTGTATGGCGTCGCCATGACCGGCATCGGCATGCTGAATCTGACCGGAAACAACGTGGCGATGGATTCCTACGGACTGATCGCCGACAATGCCAATGGAATCGGCCAAATGGCCTGGCAGGGGGAGGACGACCGCGATACGGTGCAGGCCCGCCAGATCATGGCCGACCTGGACGCGGTGGGAAACACGACCAAGGC'}

In [7]:
# let's try prompting the fine-tuned models
def continued_convo(model, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", model)
    if convo==None:
        messages=[{"role": "system", "content": """You are a helpful assistant, 
        if I ask you a question you can't answer, please reply 'I can't answer that' """}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    answer=pipe(messages, max_new_tokens=256)[0]['generated_text'][-1]
    print(answer)
    messages.append(answer)
    return pipe, messages 

In [6]:
mis_model=AutoModelForCausalLM.from_pretrained('./fine_tuned_mis/')

Loading checkpoint shards: 100%|██████| 2/2 [00:02<00:00,  1.32s/it]


In [9]:
# lla_model=AutoModelForCausalLM.from_pretrained('./fine_tuned_lla2/') 
# compatability issue, needs debugging

In [ ]:
# both of the models above are the ones pretrained using steps as the eval_strategy